In [2]:
# remove warning message
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# required library
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from local_utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import glob

In [3]:
#load pre-defined model weights and architecture for plate cropping
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [4]:
wpod_net_path = "wpod-net.json"
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [5]:
#setup for conversion
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
assert float(tf.__version__[:3]) >= 2.3

In [6]:
#convert to TF Lite
converter = tf.lite.TFLiteConverter.from_keras_model(wpod_net)

tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp7cy8ohqx/assets


In [7]:
#Convert using dynamic range quantization
converter = tf.lite.TFLiteConverter.from_keras_model(wpod_net)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()


# Save the model.
with open('plate_detect_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

INFO:tensorflow:Assets written to: /tmp/tmph0ewp2ak/assets


INFO:tensorflow:Assets written to: /tmp/tmph0ewp2ak/assets


In [8]:
# ignore warning 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import numpy as np
from PIL import Image 
import PIL
# Import the necessary libraries 
from numpy import asarray 
  

In [9]:
pwd

'/mnt/c/Users/Kyle/Desktop/Plate_Reader/Plate_detect_and_recognize'

In [10]:
dataset_paths = glob.glob("Plate_examples/*.jpg")

In [11]:
# Arange input data and corresponding labels

print(dataset_paths)
X=[]

for image_path in dataset_paths:
  image=load_img(image_path,target_size=(2000,2000))
  image=img_to_array(image)

  X.append(image)
  

X = np.array(X,dtype="float32")


print("[INFO] Find {:d} images ".format(len(X)))
print(X.shape)

['Plate_examples/china_car_plate.jpg', 'Plate_examples/china_motor_plate.jpg', 'Plate_examples/germany_car_plate.jpg', 'Plate_examples/india_car_plate.jpg', 'Plate_examples/india_motor_plate.jpg', 'Plate_examples/japan_car_plate.jpg', 'Plate_examples/japan_motor_plate.jpg', 'Plate_examples/koera_car_plate.jpg', 'Plate_examples/koera_motor_plate.jpg', 'Plate_examples/russia_car_plate.jpg', 'Plate_examples/russia_motor_plate.jpg', 'Plate_examples/saudi_car_plate.jpg', 'Plate_examples/thailand_car_plate.jpg', 'Plate_examples/thailand_motor_plate.jpg', 'Plate_examples/turkey_car_plate.jpg', 'Plate_examples/usa_car_plate.jpg', 'Plate_examples/usa_motor_plate.jpg', 'Plate_examples/vietnam_car_rectangle_plate.jpg', 'Plate_examples/vietnam_car_square_plate.jpg', 'Plate_examples/vietnam_motor_plate.jpg']
[INFO] Find 20 images 
(20, 2000, 2000, 3)


In [12]:
#convert using float fallback quantitization


def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(wpod_net)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpwatdhgwe/assets


INFO:tensorflow:Assets written to: /tmp/tmpwatdhgwe/assets


In [13]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


In [14]:

# Save the model.
with open('plate_detect_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [15]:
#Convert using Integer-only quantitization
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X).batch(1).take(100):
    yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(wpod_net)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmprb45q_kd/assets


INFO:tensorflow:Assets written to: /tmp/tmprb45q_kd/assets


RuntimeError: Quantization not yet supported for op: %

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
# Save the model.
with open('char_recognize_model_quant_int_only.tflite', 'wb') as f:
  f.write(tflite_model_quant)